In [1]:
import os
os.chdir("../")
print(os.getcwd())

c:\Users\Amrit\Intilt\Intrade_ai


In [2]:
import pandas as pd
import numpy as np
import datetime
from utils.data_cleaning import get_continuous_1min_data

In [3]:
def convert_timeframe_min(df, timeframe):
    df.index = pd.to_datetime(df.index)
    df['day'] = df.index.normalize()
    gp = df.groupby('day')
    dfList = []
    for k, res in gp:
        resampledf = res.resample(timeframe, origin='start').agg({'open':'first',
                                                        'high':'max',
                                                        'low':'min',
                                                        'close':'last'})
        resampledf.reset_index(inplace=True)
        dfList.append(resampledf)
    finaldf = pd.concat(dfList, ignore_index=True)
    finaldf = finaldf.set_index('datetime')
    return finaldf

In [4]:
data1 = pd.read_csv('Data/Reliance/reliance_1min.csv')

In [5]:
reliance_min_data = get_continuous_1min_data('Reliance',data1)

In [6]:
df1 = reliance_min_data.copy()

In [7]:
# Impute missing values using linear interpolation
df1.interpolate(method='polynomial', order=2, inplace=True)

# Add noise to the imputed values
noise = np.random.normal(0, 0.02, df1.shape)
df1 += noise

In [8]:
df1

,open,high,low,close
datetime,,,,
2011-01-03 09:15:00,531.994320,531.963821,531.616369,531.788369
2011-01-03 09:16:00,531.671916,532.446381,531.697692,532.238647
2011-01-03 09:17:00,532.159406,533.001726,532.189477,532.869657
2011-01-03 09:18:00,532.860815,532.961584,532.318551,532.399191
2011-01-03 09:19:00,532.489629,532.486919,531.800764,532.065363
...,...,...,...,...
2023-01-02 15:27:00,2577.506968,2578.004791,2576.540935,2578.016364
2023-01-02 15:28:00,2577.037042,2578.008588,2577.067743,2577.565006
2023-01-02 15:29:00,2577.973361,2577.964206,2575.935650,2577.907437


In [9]:
timeframe = '15T'
data_15_min = convert_timeframe_min(df1, timeframe)

In [10]:
data_15_min.isnull().sum()

open     20
high     20
low      20
close    20
dtype: int64

In [11]:
data_15_min[data_15_min.isna().any(axis=1)]

,open,high,low,close
datetime,,,,
2015-10-06 11:30:00,NaN,NaN,NaN,NaN
2015-10-06 11:45:00,NaN,NaN,NaN,NaN
2021-09-09 15:30:00,NaN,NaN,NaN,NaN
2021-10-18 11:30:00,NaN,NaN,NaN,NaN
2021-10-18 11:45:00,NaN,NaN,NaN,NaN
2021-10-20 15:30:00,NaN,NaN,NaN,NaN
2021-10-27 15:30:00,NaN,NaN,NaN,NaN
2021-11-08 15:30:00,NaN,NaN,NaN,NaN
2021-11-25 15:30:00,NaN,NaN,NaN,NaN


In [12]:
data_15_min = data_15_min.dropna()

In [10]:
import backtrader as bt
from datetime import datetime
import pandas as pd

class OpenRangeBreakout(bt.Strategy):
    params = (
        ("or_period", 30),
        ("atr_period", 14),
        ("stop_loss", 2),
        ("take_profit", 4),
    )

    def __init__(self):
        self.or_high = bt.indicators.RollingMax(self.data.high, period=self.params.or_period)
        self.or_low = bt.indicators.RollingMin(self.data.low, period=self.params.or_period)
        self.atr = bt.indicators.AverageTrueRange(self.data, period=self.params.atr_period)

    def next(self):
        if not self.position:
            if self.data.high[0] > self.or_high[0]:
                self.buy(stop=self.or_high[0])
            elif self.data.low[0] < self.or_low[0]:
                self.sell(stop=self.or_low[0])
        else:
            if self.data.low[0] < self.data.close[-1] - self.params.stop_loss * self.atr[0]:
                self.close()
            elif self.data.high[0] > self.data.close[-1] + self.params.take_profit * self.atr[0]:
                self.close()



cerebro = bt.Cerebro()
data = bt.feeds.YahooFinanceCSVData(
        dataname='Data/Nifty/nifty_1day_continous.csv',
        # Do not pass values before this date
        fromdate=datetime(2007, 1, 3),
        # Do not pass values before this date
        todate=datetime(2007, 3, 31),
        # Do not pass values after this date
        reverse=False)
cerebro.adddata(data)
cerebro.addstrategy(OpenRangeBreakout)
cerebro.run()
data = cerebro.plot()
data[0][0].savefig("data.png")

AttributeError: module 'backtrader.indicators' has no attribute 'RollingMax'

In [12]:
import backtrader as bt
import pandas as pd
from datetime import date, datetime, time, timedelta
import matplotlib

class OpeningRangeBreakout(bt.Strategy):
    params = dict(
        num_opening_bars=3,
        fast_length= 5*12*6,
        slow_length= 25*12*6
    )

    def __init__(self):
        self.bought_today = False
        self.order = None
        self.crossovers = []
        self.opening_range_low = {}
        self.opening_range_high = {}
        self.bought_today = {}
        self.opening_range = {}
        
           
    def log(self, txt, dt=None):
        if dt is None:
            dt = self.datas[0].datetime.datetime()

        # print('%s, %s' % (dt, txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            order_details = f"{order.executed.price}, Cost: {order.executed.value}, Comm {order.executed.comm}"

            if order.isbuy():
                self.log(f"BUY EXECUTED, Price: {order_details}")
            else:  # Sell
                self.log(f"SELL EXECUTED, Price: {order_details}")

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        for i, d in enumerate(self.datas):
            current_bar_datetime = d.num2date(d.datetime[0])
            previous_bar_datetime = d.num2date(d.datetime[-1])
            print("current_bar_datetime",current_bar_datetime, 
                  "previous_bar_datetime", previous_bar_datetime)

            if (current_bar_datetime.date() != previous_bar_datetime.date()) :
                self.opening_range_low[i] = d.low[0]
                self.opening_range_high[i] = d.high[0]
                self.bought_today[i] = False

            opening_range_start_time = time(9, 30, 0)
            dt = datetime.combine(date.today(), opening_range_start_time) + \
                timedelta(minutes=self.p.num_opening_bars * 5)
            opening_range_end_time = dt.time()

            print("self.opening_range_high",self.opening_range_high)
            
            if (current_bar_datetime.time() >= opening_range_start_time) \
                    and (current_bar_datetime.time() < opening_range_end_time):
                self.opening_range_high[i] = max(d.high[0], self.opening_range_high[i])
                self.opening_range_low[i] = min(d.low[0], self.opening_range_low[i])
                self.opening_range[i] = self.opening_range_high[i] - self.opening_range_low[i]
            else:

                if self.order:
                    return

                if self.getposition(d).size and (d.close[0] > (self.opening_range_high[i] + self.opening_range[i])):
                    self.close()

                if d.close[0] > self.opening_range_high[i] and not self.getposition(d).size and not self.bought_today[i]:
                    self.order = self.buy()
                    self.bought_today[i] = True

                if self.getposition(d).size and (self.data.close[0] < (self.opening_range_high[i] - self.opening_range[i])):
                    self.order = self.close()

                if self.getposition(d).size and current_bar_datetime.time() >= time(15, 45, 0):
                    self.log("RUNNING OUT OF TIME - LIQUIDATING POSITION")
                    self.close()

    def stop(self):
        self.roi = (self.broker.get_value() / 100000) - 1.0


if __name__ == '__main__':

    cerebro = bt.Cerebro()
    data = bt.feeds.YahooFinanceCSVData(
            dataname='Data/Reliance/reliance_1min.csv',
            # Do not pass values before this date
            fromdate=datetime(2007, 1, 2),
            # Do not pass values before this date
            todate=datetime(2007, 3, 31),
            # Do not pass values after this date
            reverse=False)
    cerebro.adddata(data)
    initial_cash = 100000.0
    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=0.0)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=95)
    cerebro.addwriter(bt.WriterFile, csv=True)
    cerebro.addstrategy(OpeningRangeBreakout)

    strats = cerebro.optstrategy(OpeningRangeBreakout, num_opening_bars=[15, 30, 60])
    cerebro.run()

    # Get final portfolio Value
    portvalue = cerebro.broker.getvalue()

    # Print out the final result
    print('Final Portfolio Value: ${}'.format(portvalue))